In [1]:
import pandas as pd
df = pd.read_excel('combined_news.xlsx')
#drop empty data
df = df.dropna(subset=['Content'])
df = df.dropna(subset=['Title'])

In [3]:
df

,Title,Tag,Created Date,url,Content
0,A fire left Lucy Yu’s literary hub in Chinatow...,"['GOOGL', 'AMZN']",2024-06-01T09:01:40+0000,https://www.nytimes.com/2024/06/01/business/yu...,Lucy Yu wasn’t sure if she had smoke in her lu...
1,"Since Google overhauled its search engine, pub...","['GOOGL', 'MSFT']",2024-06-01T09:02:28+0000,https://www.nytimes.com/2024/06/01/technology/...,When Frank Pine searched Google for a link to ...
2,Bhutanese Americans are the newest citizens of...,['AAPL'],2024-06-01T09:02:59+0000,https://www.nytimes.com/2024/06/01/us/asian-am...,When Dinesh Nepal first arrived in the United ...
3,She co-starred in the Apple TV+ show with Coli...,['AAPL'],2024-06-01T09:03:39+0000,https://www.nytimes.com/2024/06/01/arts/televi...,When Kirby accepted the role of Ruby — a Bondi...
4,"“The Boys,” “Orphan Black: Echoes,” “The Bear”...","['AMZN', 'AAPL']",2024-06-01T09:03:58+0000,https://www.nytimes.com/2024/06/01/arts/televi...,"Every month, streaming services add movies and..."
...,...,...,...,...,...
890,"Hear triumphant tracks from John Williams, Whi...",['AAPL'],2024-07-30T18:00:02+0000,https://www.nytimes.com/2024/07/30/arts/music/...,"I just got out my thermometer to confirm and, ..."
891,The Amazon Labor Union has been divided over s...,['AMZN'],2024-07-30T19:47:43+0000,https://www.nytimes.com/2024/07/30/business/ec...,A dissident group has won control of the Amazo...
892,"The tech giant’s revenue also grew 15 percent,...","['GOOGL', 'MSFT']",2024-07-30T20:51:51+0000,https://www.nytimes.com/2024/07/30/technology/...,Microsoft closed its first full fiscal year of...
893,The city’s leaders are forged in a mix of weal...,['AAPL'],2024-07-30T22:29:22+0000,https://www.nytimes.com/2024/07/30/us/politics...,Good evening. The presidential race has become...


In [5]:
from transformers import pipeline
import nltk
# 初始化 FinBERT 模型
classifier = pipeline("sentiment-analysis", model="yiyanghkust/finbert-tone", tokenizer="yiyanghkust/finbert-tone")

def compute_finbert_sentiment(text, method="avg"):
    if method == "avg":
        try:
            chunks = nltk.sent_tokenize(text)
            #print(chunks,len(chunks))
            sentiments = []
            for chunk in chunks:
                result = classifier(chunk)
                # print(result)
                label = result[0]['label']
                score = result[0]['score']
                # 将标签转换为情感分数并添加到列表中
                if label == 'Positive':
                    sentiments.append(score*1)
                elif label == 'Neutral':
                    sentiments.append(0)
                elif label == 'Negative':
                    sentiments.append(score*-1)
            score = sum(sentiments) / len(sentiments)
            if score > 0:
                positive_values = [val for val in sentiments if val > 0]
                return  sum(positive_values) / len(positive_values)
            elif score < 0:
                negative_values = [val for val in sentiments if val < 0]
                return sum(negative_values) / len(negative_values)
            else:
                return 0
        except Exception as e:
            print(f"Error processing text: {e}")
        return 0.0
    if method == "max":
        try:
            chunks = nltk.sent_tokenize(text)
            #print(chunks,len(chunks))
            sentiments = []
            for chunk in chunks:
                result = classifier(chunk)
                # print(result)
                label = result[0]['label']
                score = result[0]['score']
                # 将标签转换为情感分数并添加到列表中
                if label == 'Positive':
                    sentiments.append(score*1)
                elif label == 'Neutral':
                    sentiments.append(0)
                elif label == 'Negative':
                    sentiments.append(score*-1)
            score = sum(sentiments) / len(sentiments)
            if score > 0:
                return max(sentiments)
            elif score < 0:
                return min(sentiments)
            else:
                return 0
        except Exception as e:
            print(f"Error processing text: {e}")
        return 0.0

# 处理 DataFrame 中的文本
df['sentiment_title'] = df['Title'].apply(compute_finbert_sentiment)
df

Some layers from the model checkpoint at yiyanghkust/finbert-tone were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at yiyanghkust/finbert-tone.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.
C:\Users\matth\anaconda3\envs\huggingface-tensorflow\lib\site

,Title,Tag,Created Date,url,Content,sentiment_title
0,A fire left Lucy Yu’s literary hub in Chinatow...,"['GOOGL', 'AMZN']",2024-06-01T09:01:40+0000,https://www.nytimes.com/2024/06/01/business/yu...,Lucy Yu wasn’t sure if she had smoke in her lu...,-0.999550
1,"Since Google overhauled its search engine, pub...","['GOOGL', 'MSFT']",2024-06-01T09:02:28+0000,https://www.nytimes.com/2024/06/01/technology/...,When Frank Pine searched Google for a link to ...,-0.877579
2,Bhutanese Americans are the newest citizens of...,['AAPL'],2024-06-01T09:02:59+0000,https://www.nytimes.com/2024/06/01/us/asian-am...,When Dinesh Nepal first arrived in the United ...,0.000000
3,She co-starred in the Apple TV+ show with Coli...,['AAPL'],2024-06-01T09:03:39+0000,https://www.nytimes.com/2024/06/01/arts/televi...,When Kirby accepted the role of Ruby — a Bondi...,0.000000
4,"“The Boys,” “Orphan Black: Echoes,” “The Bear”...","['AMZN', 'AAPL']",2024-06-01T09:03:58+0000,https://www.nytimes.com/2024/06/01/arts/televi...,"Every month, streaming services add movies and...",0.000000
...,...,...,...,...,...,...
890,"Hear triumphant tracks from John Williams, Whi...",['AAPL'],2024-07-30T18:00:02+0000,https://www.nytimes.com/2024/07/30/arts/music/...,"I just got out my thermometer to confirm and, ...",0.000000
891,The Amazon Labor Union has been divided over s...,['AMZN'],2024-07-30T19:47:43+0000,https://www.nytimes.com/2024/07/30/business/ec...,A dissident group has won control of the Amazo...,0.000000
892,"The tech giant’s revenue also grew 15 percent,...","['GOOGL', 'MSFT']",2024-07-30T20:51:51+0000,https://www.nytimes.com/2024/07/30/technology/...,Microsoft closed its first full fiscal year of...,0.000000
893,The city’s leaders are forged in a mix of weal...,['AAPL'],2024-07-30T22:29:22+0000,https://www.nytimes.com/2024/07/30/us/politics...,Good evening. The presidential race has become...,0.998038


In [6]:
df['sentiment_content'] = df['Content'].apply(compute_finbert_sentiment)
df

,Title,Tag,Created Date,url,Content,sentiment_title,sentiment_content
0,A fire left Lucy Yu’s literary hub in Chinatow...,"['GOOGL', 'AMZN']",2024-06-01T09:01:40+0000,https://www.nytimes.com/2024/06/01/business/yu...,Lucy Yu wasn’t sure if she had smoke in her lu...,-0.999550,0.0
1,"Since Google overhauled its search engine, pub...","['GOOGL', 'MSFT']",2024-06-01T09:02:28+0000,https://www.nytimes.com/2024/06/01/technology/...,When Frank Pine searched Google for a link to ...,-0.877579,0.0
2,Bhutanese Americans are the newest citizens of...,['AAPL'],2024-06-01T09:02:59+0000,https://www.nytimes.com/2024/06/01/us/asian-am...,When Dinesh Nepal first arrived in the United ...,0.000000,0.0
3,She co-starred in the Apple TV+ show with Coli...,['AAPL'],2024-06-01T09:03:39+0000,https://www.nytimes.com/2024/06/01/arts/televi...,When Kirby accepted the role of Ruby — a Bondi...,0.000000,0.0
4,"“The Boys,” “Orphan Black: Echoes,” “The Bear”...","['AMZN', 'AAPL']",2024-06-01T09:03:58+0000,https://www.nytimes.com/2024/06/01/arts/televi...,"Every month, streaming services add movies and...",0.000000,0.0
...,...,...,...,...,...,...,...
890,"Hear triumphant tracks from John Williams, Whi...",['AAPL'],2024-07-30T18:00:02+0000,https://www.nytimes.com/2024/07/30/arts/music/...,"I just got out my thermometer to confirm and, ...",0.000000,0.0
891,The Amazon Labor Union has been divided over s...,['AMZN'],2024-07-30T19:47:43+0000,https://www.nytimes.com/2024/07/30/business/ec...,A dissident group has won control of the Amazo...,0.000000,0.0
892,"The tech giant’s revenue also grew 15 percent,...","['GOOGL', 'MSFT']",2024-07-30T20:51:51+0000,https://www.nytimes.com/2024/07/30/technology/...,Microsoft closed its first full fiscal year of...,0.000000,-1.0
893,The city’s leaders are forged in a mix of weal...,['AAPL'],2024-07-30T22:29:22+0000,https://www.nytimes.com/2024/07/30/us/politics...,Good evening. The presidential race has become...,0.998038,0.0


In [7]:
df.to_csv('output/finbert_avg.csv', index=False, encoding='utf-8-sig')